### Implementing Neural Network Using Pytorch

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
dtype = torch.float
device = torch.device('cuda:0')

In [3]:
N, D_in, H, D_out = 64, 1000, 100, 10

In [9]:
%%time
x = torch.randn(N, D_in, device=device, dtype = dtype)
y = torch.randn(N, D_out, device=device, dtype = dtype)

CPU times: user 411 µs, sys: 152 µs, total: 563 µs
Wall time: 296 µs


In [10]:
w1 = torch.rand(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.rand(H, D_out, device=device, dtype=dtype, requires_grad=True)

In [11]:
learning_rate = 1e-6

In [12]:
#Forward Pass

for i in range(500):
    #Predict
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    # Compute Loss
    loss = (y_pred - y).pow(2).sum()
    
    if i%100 == 99:
        print(i, loss.item())
        
    #backprop
    loss.backward()
    
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        w1.grad.zero_()
        w2.grad.zero_()

99 8943.228515625
199 2844.3408203125
299 1418.90625
399 878.806396484375
499 616.8677368164062


### Pytorch - Building Neural Network Using NN

In [20]:
N, D_in, H, D_out = 64, 1000, 100, 10
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

In [21]:
model = nn.Sequential(
        nn.Linear(D_in, H),
        nn.ReLU(),
        nn.Linear(H, D_out)
        )

In [22]:
loss_fn = nn.MSELoss(reduction='sum')

In [24]:
learning_rate = 1e-4
for t in range(500):
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    
    if t%100 == 99:
        print(t, loss.item())
    
    model.zero_grad()
    loss.backward()
    
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate*param.grad
            
##### one can also use the parameter update using optim module in nn. 
##### https://pytorch.org/tutorials/beginner/pytorch_with_examples.html

99 1.717417376312369e-06
199 2.0345160578472132e-07
299 2.812593002943231e-08
399 5.959012838729905e-09
499 2.2485828754525983e-09


### Creating Custom Modules Using NN

In [28]:
class TwoLayerNetwork(nn.Module):
    def __init__(self, D_in, H, D_out):
        super(TwoLayerNetwork, self).__init__()
        self.Linear1 = nn.Linear(D_in, H)
        self.Linear2 = nn.Linear(H, D_out)
        
    def forward(self, x):
        relu = nn.ReLU()
        h_relu = relu(self.Linear1(x))
        y_pred = self.Linear2(h_relu)
        
        return y_pred
    
N, D_in, H, D_out = 64, 1000, 100, 10
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = TwoLayerNetwork(D_in, H, D_out)

In [29]:
criterion = torch.nn.MSELoss(reduce='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

/opt/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [30]:
for t in range(500):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 1.0673742294311523
199 1.0547248125076294
299 1.0423656702041626
399 1.0302541255950928
499 1.0183964967727661


### PyTorch: Control Flow + Weight Sharing

In [34]:
import random

In [39]:
class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):

        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)

    def forward(self, x):
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred
#### Creating Dynamic number of Hidden Layers    

In [40]:
N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = DynamicNet(D_in, H, D_out)

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
    y_pred = model(x)

    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 40.280517578125
199 0.6528862118721008
299 0.538093626499176
399 0.26977047324180603
499 0.3849489390850067
